<a href="https://colab.research.google.com/github/hatefap/CrackingTheInterview/blob/master/mbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip  install transformers numpy torch sklearn

     |████████████████████████████████| 2.0MB 5.5MB/s 
     |████████████████████████████████| 890kB 18.8MB/s 
     |████████████████████████████████| 3.2MB 25.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=ac8bea640d67dd33584aacad8c50f105e16bbdb11e933968322c61ff5c403a9f
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import torch
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pandas as pd

In [ ]:
def set_seed(seed: int):
    """
    Helper function for reproducible behavior to set the seed in ``random``, ``numpy``, ``torch`` and/or ``tf`` (if
    installed).

    Args:
        seed (:obj:`int`): The seed to set.
    """
    random.seed(seed)
    np.random.seed(seed)
    if is_torch_available():
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        # ^^ safe to call this function even if cuda is not available
    if is_tf_available():
        import tensorflow as tf

        tf.random.set_seed(seed)

set_seed(101)

In [ ]:
model_name = "bert-base-multilingual-uncased"
# max sequence length for each document/sentence sample
max_length = 512

In [ ]:
tokenizer = BertTokenizer.from_pretrained(model_name, do_lower_case=True)

In [ ]:
farsi_path = "drive/MyDrive/final_final.csv"
english_path = "drive/MyDrive/my_final_english_hate.csv"
eng = pd.read_csv(english_path)
farsi = pd.read_csv(farsi_path)
eng.dropna(inplace=True)
farsi.dropna(inplace=True)

In [ ]:
X_train = eng[['tweet']]
Y_train = eng[['class']]


X_farsi = farsi[['text']]
Y_farsi = farsi[['ref3']]

In [ ]:
X_tune, X_test, Y_tune, Y_test = train_test_split(X_farsi, Y_farsi, test_size=0.4, random_state=101)

In [ ]:
# tokenize the dataset, truncate when passed `max_length`, 
# and pad with 0's when less than `max_length`
train_encodings = tokenizer(X_train['tweet'].to_list(), truncation=True, padding=True, max_length=max_length)
farsi_valid_encodings = tokenizer(X_tune['text'].to_list(), truncation=True, padding=True, max_length=max_length)

In [ ]:
class toTorchConverter(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)

# convert our tokenized data into a torch Dataset
train_dataset = toTorchConverter(train_encodings, Y_train['class'].to_list())
farsi_dataset_valid = toTorchConverter(farsi_valid_encodings, Y_tune['ref3'].astype(dtype="long").to_list())

In [ ]:
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2).to("cuda")

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

In [ ]:
from sklearn.metrics import accuracy_score

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  # calculate accuracy using sklearn's function
  acc = accuracy_score(labels, preds)
  return {
      'accuracy': acc,
  }

In [ ]:
training_args = TrainingArguments(
    output_dir='./results_mbert',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=20,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs_mbert',            # directory for storing logs
    load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)
    # but you can specify `metric_for_best_model` argument to change to accuracy or other metric
    logging_steps=400,               # log & save weights each logging_steps
    evaluation_strategy="steps",     # evaluate each `logging_steps`
)

In [ ]:
trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=farsi_dataset_valid,          # evaluation dataset
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
)

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,Runtime,Samples Per Second
400,0.532700,0.607153,0.669769,73.653600,42.306000
800,0.407600,0.537932,0.756098,73.761600,42.244000
1200,0.365300,0.576472,0.850770,73.633400,42.318000
1600,0.317100,0.922639,0.626765,73.694500,42.283000
2000,0.255400,0.656440,0.745828,73.558500,42.361000


TrainOutput(global_step=2037, training_loss=0.3738841902817589, metrics={'train_runtime': 3670.0503, 'train_samples_per_second': 0.555, 'total_flos': 9350497083274776.0, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': 320823, 'init_mem_gpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 18306, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 1900191, 'train_mem_gpu_alloc_delta': 2725627392, 'train_mem_cpu_peaked_delta': 326288863, 'train_mem_gpu_peaked_delta': 4685679616})

In [ ]:
trainer.evaluate()

{'epoch': 3.0,
 'eval_accuracy': 0.7560975609756098,
 'eval_loss': 0.5379319190979004,
 'eval_mem_cpu_alloc_delta': 124972,
 'eval_mem_cpu_peaked_delta': 219606,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_gpu_peaked_delta': 159190016,
 'eval_runtime': 73.6702,
 'eval_samples_per_second': 42.297}

In [ ]:
model_path = "drive/MyDrive/mbert"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

('drive/MyDrive/mbert/tokenizer_config.json',
 'drive/MyDrive/mbert/special_tokens_map.json',
 'drive/MyDrive/mbert/vocab.txt',
 'drive/MyDrive/mbert/added_tokens.json')

In [ ]:
def get_prediction(text):
    # prepare our text into tokenized sequence
    inputs = tokenizer(text, padding=True, truncation=True, max_length=max_length, return_tensors="pt").to("cuda")
    # perform inference to our model
    outputs = model(**inputs)
    # get output probabilities by doing softmax
    probs = outputs[0].softmax(1)
    # executing argmax function to get the candidate label
    return probs.argmax().item()

In [ ]:
X_test_result = [get_prediction(item) for item in X_test['text'].to_list()]


In [ ]:
print(classification_report(Y_test, X_test_result))

              precision    recall  f1-score   support

         0.0       0.88      0.82      0.84      1786
         1.0       0.20      0.29      0.24       292

    accuracy                           0.74      2078
   macro avg       0.54      0.55      0.54      2078
weighted avg       0.78      0.74      0.76      2078



In [ ]:
model_without_english = BertForSequenceClassification.from_pretrained(model_name, num_labels=2).to("cuda")

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

In [ ]:
!rm -rf results_mbert/*

In [ ]:
training_args_without_english = TrainingArguments(
    output_dir='./mbert_without_english',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=20,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./mbert_without_english_logs',            # directory for storing logs
    load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)
    # but you can specify `metric_for_best_model` argument to change to accuracy or other metric
    logging_steps=400,               # log & save weights each logging_steps
    evaluation_strategy="steps",     # evaluate each `logging_steps`
)

In [ ]:
trainer_without_english = Trainer(
    model=model_without_english,                         # the instantiated Transformers model to be trained
    args=training_args_without_english,                  # training arguments, defined above
    train_dataset=farsi_dataset_valid,         # training dataset
    eval_dataset=farsi_dataset_valid,
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
)

In [ ]:
trainer_without_english.train()

Step,Training Loss,Validation Loss,Accuracy,Runtime,Samples Per Second
400,0.407000,0.262284,0.903402,73.650000,42.308000


TrainOutput(global_step=585, training_loss=0.35791881593883546, metrics={'train_runtime': 730.5216, 'train_samples_per_second': 0.801, 'total_flos': 1821033947246688.0, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': 49382, 'init_mem_gpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 18306, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 976665, 'train_mem_gpu_alloc_delta': 2684697088, 'train_mem_cpu_peaked_delta': 325453922, 'train_mem_gpu_peaked_delta': 2435593728})

In [ ]:
trainer_without_english.evaluate()

{'epoch': 3.0,
 'eval_accuracy': 0.9034017971758665,
 'eval_loss': 0.2622841000556946,
 'eval_mem_cpu_alloc_delta': 120903,
 'eval_mem_cpu_peaked_delta': 225581,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_gpu_peaked_delta': 159853568,
 'eval_runtime': 73.7237,
 'eval_samples_per_second': 42.266}

In [ ]:
def get_prediction_wighout_english(text):
    # prepare our text into tokenized sequence
    inputs = tokenizer(text, padding=True, truncation=True, max_length=max_length, return_tensors="pt").to("cuda")
    # perform inference to our model
    outputs = model_without_english(**inputs)
    # get output probabilities by doing softmax
    probs = outputs[0].softmax(1)
    # executing argmax function to get the candidate label
    return probs.argmax().item()

In [ ]:
X_test_result = [get_prediction_wighout_english(item) for item in X_test['text'].to_list()]

print(classification_report(Y_test, X_test_result))

              precision    recall  f1-score   support

         0.0       0.91      0.96      0.93      1786
         1.0       0.63      0.44      0.52       292

    accuracy                           0.88      2078
   macro avg       0.77      0.70      0.73      2078
weighted avg       0.87      0.88      0.88      2078

